# TED Talks Öneri Sistemi | Sentence-BERT Recommendation

| Özellik | Detay |
|---|---|
| **Görev** | Content-Based Konuşma Önerisi |
| **Model** | Sentence-BERT — all-MiniLM-L6-v2 (~80MB, 384-dim embeddings) |
| **Veri** | ted_main.csv + transcripts.csv — 2467 TED konuşması |
| **Benzerlik** | Cosine Similarity (2467 × 2467 matris) |
| **Çıktı** | Seçilen konuşmaya anlamsal olarak en yakın Top-10 öneri |
| **Teknoloji** | sentence-transformers, scikit-learn, Pandas, joblib |

**Avantaj:** TF-IDF'in aksine kelime sıklığı değil **anlamsal benzerlik** ölçülür — "iklim değişikliği" ve "küresel ısınma" gibi eş anlamlı ifadeler doğru eşleştirilir.
**Pipeline:** Transkript metni → SBERT Encoding (384-dim) → Cosine Similarity Matrisi → Top-N Öneri

In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import os


## Veri Yükleme (Data Loading)
  Gerekli kütüphaneler (sentence_transformers, pandas, sklearn) yüklenir. ted_main.csv (meta veriler) ve transcripts.csv (metin
  dökümleri) dosyaları okunur ve url sütunu üzerinden birleştirilir. Analiz için sadece başlık, konuşmacı ve metin sütunları seçilir.
  Eksik metin verileri temizlenir.

In [8]:
df_main = pd.read_csv('ted_main.csv',encoding='latin-1')
df_transcripts = pd.read_csv('transcripts.csv',encoding='latin-1')
df = pd.merge(df_main, df_transcripts, on='url')
df = df[['title', 'transcript', 'url', 'main_speaker']]
df['transcript'] = df['transcript'].fillna('')
print(df.shape)


(2542, 4)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         2542 non-null   object
 1   transcript    2542 non-null   object
 2   url           2542 non-null   object
 3   main_speaker  2542 non-null   object
dtypes: object(4)
memory usage: 79.6+ KB


In [10]:
df.head()

,title,transcript,url,main_speaker
0,Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,Ken Robinson
1,Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,Al Gore
2,Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,David Pogue
3,Greening the ghetto,If you're here today _ÑÓ and I'm very happy th...,https://www.ted.com/talks/majora_carter_s_tale...,Majora Carter
4,The best stats you've ever seen,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,Hans Rosling


## Deep Learning Modelinin Yüklenmesi
  Hugging Face kütüphanesinden all-MiniLM-L6-v2 modeli indirilir. Bu model, cümleleri ve paragrafları hızlı ve etkili bir şekilde
  vektörleştirmek için optimize edilmiştir. Boyutu küçük olmasına rağmen (yaklaşık 80MB), performansı oldukça yüksektir.

###  Metinlerin Vektörleştirilmesi (Encoding)
  Veri setindeki tüm TED konuşma metinleri (transcripts), yüklenen Transformer modelinden geçirilir. Model, her bir konuşma için 384
  sayıdan oluşan benzersiz bir Embedding Vektörü üretir. Bu vektörler, konuşmanın "anlamsal parmak izi" gibidir. İşlem sonunda
  elimizde (Konuşma Sayısı, 384) boyutunda bir matris oluşur.

### Benzerlik Matrisi (Cosine Similarity)
  Elde edilen embedding matrisi kullanılarak, her konuşmanın diğer tüm konuşmalarla olan benzerlik skoru hesaplanır. Sonuçta (N, N)
  boyutunda kare bir matris elde edilir. Bu matrisin [i, j] hücresi, i. ve j. konuşmaların ne kadar benzer olduğunu (0 ile 1
  arasında) gösterir.

### Öneri Testi
  Sistemin çalışıp çalışmadığını test etmek için rastgele bir konuşma seçilir. Hesaplanan benzerlik matrisinden bu konuşmaya ait
  satır çekilir, skorlar büyükten küçüğe sıralanır ve en yüksek skora sahip ilk 10 konuşma (kendisi hariç) listelenir.

In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['transcript'].tolist(), show_progress_bar=True)
cosine_sim_dl = cosine_similarity(embeddings, embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Yusuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yusuf\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

## Modelleri Kaydetme
  Oluşturulan Benzerlik Matrisi (cosine_sim_dl.pkl), Veri Seti (ted_data.pkl) ve İndeks Sözlüğü (indices.pkl) diske kaydedilir. Bu
  dosyalar, modelin tekrar tekrar eğitilmesine gerek kalmadan Streamlit uygulamasında hızlıca kullanılmasını sağlar.

In [8]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(cosine_sim_dl, '../models/cosine_sim_dl.pkl')
joblib.dump(df[['title', 'main_speaker', 'url']], '../models/ted_data.pkl')
joblib.dump(indices, '../models/indices.pkl')
print('Saved.')


Saved.


## Sonuç ve Değerlendirme

| Bileşen | Detay |
|---|---|
| **Veri** | 2467 TED konuşması (ted_main + transcripts birleşimi) |
| **Model** | all-MiniLM-L6-v2 — 384 boyutlu anlamsal vektörler |
| **Benzerlik** | Cosine Similarity — 2467 × 2467 matris |
| **Öneri** | Top-10 anlamsal olarak en yakın konuşma (kendisi hariç) |
| **Çıktı Dosyaları** | cosine_sim_dl.pkl · ted_data.pkl · indices.pkl |

**Bulgular:**
- Sentence-BERT, kelime örtüşmesi yerine derin anlamsal bağlamı yakaladığından TF-IDF'e göre daha kaliteli öneriler üretmektedir
- all-MiniLM-L6-v2 (~80MB), büyük Transformer modellerine kıyasla hızlı ve hafif bir alternatif sunmaktadır
- 384 boyutlu embedding vektörleri tam transkript metnini temsil ettiğinden zengin içerik bilgisi sağlanmaktadır
- Sistem bir kez eğitilip kaydedildikten sonra Streamlit uygulamasında gerçek zamanlı öneri için kullanılabilir
- Konuşma etiketleri (tags), görüntülenme sayısı veya konuşmacı bilgisi eklenerek hibrit öneri sistemi geliştirilebilir